In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!./Miniconda3-latest-Linux-x86_64.sh -b -p /usr/local -f

In [2]:
! conda install -c milagraph -c conda-forge graphvite python=3.7 cudatoolkit=10.1 -y 

| / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / 

In [ ]:
#!conda install wurlitzer ipykernel -y
! pip install wurlitzer
! pip install ipykernel

In [6]:
import site
site.addsitedir("/usr/local/lib/python3.7/site-packages")
%reload_ext wurlitzer

import torch
from graphvite.application.network import LinkPredictor

In [28]:
def get_score(model, H, T):
    '''
    Get the un-normalized score (similarity) for `H` and `T` 
    within the given `model`
    '''
    model = LinkPredictor('DeepWalk', model.solver.vertex_embeddings, model.solver.context_embeddings)
    model = model
    tensorH = torch.as_tensor(H)
    tensorT = torch.as_tensor(T)
    return model(tensorH, tensorT)

In [12]:
cd drive/MyDrive/Graph_Link_Prediction/graphvite/

/content/drive/MyDrive/Graph_Link_Prediction/graphvite


In [26]:
! graphvite run my_config.yml

loading graph from data/edges.txt
0.00106795%
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
Graph<uint32>
------------------ Graph -------------------
#vertex: 3816, #edge: 59294
as undirected: yes, normalization: no
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
[time] GraphApplication.load: 0.0169356 s
[time] GraphApplication.build: 0.303398 s
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
GraphSolver<32, float32, uint32>
----------------- Resource -----------------
#worker: 1, #sampler: 1, #partition: 1
tied weights: no, episode size: 200
gpu memory limit: 11.1 GiB
gpu memory cost: 42.2 MiB
----------------- Sampling -----------------
augmentation step: 5
random walk length: 40
random walk batch size: 100
#negative: 1, negative sample exponent: 0.75
----------------- Training -----------------
model: DeepWalk
optimizer: SGD
learning rate: 0.025, lr schedule: linear
weight decay: 0.005
#epoch: 4000, batch size: 100000
resume: no
positive reuse: 1, negative weight: 5
>>>>>>>>>>>>>>>>>>>>>>

In [27]:
import pickle
emb = 'model/LINE_full.pkl'
model = pickle.load(open(emb, 'rb'))

In [30]:
H, T = [], []
mapping = model.graph.name2id
with open('data/test-public.csv', 'r') as f:
    next(f)
    for line in f:
        _, h, t = line.split(',')
        h = str(int(h))
        t = str(int(t))
        H.append(mapping[h])
        T.append(mapping[t])

In [31]:
score = get_score(model, H, T)
# normalize the scores to probabilities
m = torch.nn.Sigmoid()
score = m(score)


In [32]:
print(score)

tensor([0.6023, 0.0332, 0.0391,  ..., 0.1025, 0.8993, 0.0230])


In [33]:
# save the output
out = 'your_name.csv'
with open(out, 'w') as o:
    o.write('Id,Predicted\n')
    for i, s in enumerate(score.tolist()):
        o.write(f'{i+1},{s}\n')
print('Write output to', out)

Write output to your_name.csv
